In [1]:
using Random
using DataFrames
using CSV
using Statistics

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 100
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [5]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    while budget_sofar > budget
        hf_ratio = rand() * .70 + rand() * 0.03 + 0.02
        l_ratio = rand()
        h_temp = rand() + 0.15
        f_temp = rand() + 0.15
        if hf_ratio > 0.05
            while abs(h_temp - f_temp) < 0.3
                h_temp = rand() + 0.15
                f_temp = rand() + 0.15
            end
        end
        budget_hf = (0.05 + hf_ratio * 0.8) * (budget * .75)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 5 * l_ratio)/(h^2+f^2 + 5)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter("No Name [Scheme A]", h, f, l, t)
end

#evaluate_generator(random_scheme_a, 1000, 50)

random_scheme_a (generic function with 6 methods)

In [6]:
evaluate_generator(random_scheme_a, 1000, 50)

-0.35916000000000003

In [7]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0

hf_ratio = rand()
l_ratio = rand()
h_temp = rand()
f_temp = rand()
budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
l_ratio2 = (h-2)^2/((h-2)^2+(f-2)^2)
#l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
lraw = l0 + ( budget_remain * l_ratio2 )/h
traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end

println((h, f, l, t, budget_hf, budget_sofar, hf_ratio))

(12.0, 137.0, 22.0, 59.0, 1644.0, 9883.0, 0.16012703389256688)


In [8]:
# budget = MXS
# h0 = MNV
# f0 = MNV
# l0 = MNV
# t0 = MNV
# hf_ratio0 = 0
# l_ratio0 = 0
# h_temp0 = 0
# f_temp0 = 0


# for i in 1:1000
#     hf_ratio = rand()
#     l_ratio = 0.25 + 0.5 * rand()
#     h_temp = rand() + 0.1
#     f_temp = rand() + 0.1
#     budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
#     h_ratio = h_temp/sqrt(h_temp * f_temp)
#     f_ratio = f_temp/sqrt(h_temp * f_temp)
#     h = max(h0, round(sqrt(budget_hf) * h_ratio))
#     f = max(f0, round(sqrt(budget_hf) * f_ratio))
#     budget_hf = h * f
#     budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
#     lraw = l0 + ( budget_remain * l_ratio )/h
#     traw = t0 + ( budget_remain * (1 - l_ratio) )/f
#     l = trunc(lraw)
#     t = trunc(traw)
#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         could_add_l = (budget - budget_sofar) >= h
#         could_add_t = (budget - budget_sofar) >= f
#         if could_add_l && !could_add_t
#             l = l+trunc((budget - budget_sofar)/h)
#         end
#         if !could_add_l && could_add_t
#             t = t+trunc((budget - budget_sofar)/f)
#         end
#         if could_add_l && could_add_t
#             if fpart(lraw) > fpart(traw)
#                 l = l+trunc((budget - budget_sofar)/h)
#             else
#                 t = t+trunc((budget - budget_sofar)/f)
#             end
#         end
#     end

#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         hf_ratio0 = hf_ratio
#         l_ratio0 = l_ratio
#         h_temp0 = h_temp
#         f_temp0 = f_temp
#         println((h, f, l, t, budget_hf, budget_sofar))
#     end
# end

In [9]:
# hf_ratio = hf_ratio0
# l_ratio = l_ratio0
# h_temp = h_temp0
# f_temp = f_temp0

In [10]:
# budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
# h_ratio = h_temp/sqrt(h_temp * f_temp)
# f_ratio = f_temp/sqrt(h_temp * f_temp)
# h = max(h0, round(sqrt(budget_hf) * h_ratio))
# f = max(f0, round(sqrt(budget_hf) * f_ratio))
# budget_hf = h * f
# budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
# lraw = l0 + ( budget_remain * l_ratio )/h
# traw = t0 + ( budget_remain * (1 - l_ratio) )/f
# l = trunc(lraw)
# t = trunc(traw)
# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

In [11]:
# if (budget - budget_sofar) >= min(h, f)
#     could_add_l = (budget - budget_sofar) >= h
#     could_add_t = (budget - budget_sofar) >= f
#     if could_add_l && !could_add_t
#         l = l+1
#     end
#     if !could_add_l && could_add_t
#         t = t+1
#     end
#     if could_add_l && could_add_t
#         if fpart(lraw) > fpart(traw)
#             l = l+1
#         else
#             t = t+1
#         end
#     end
# end

# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

## Testing Random Scheme A

In [12]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("No Name [Scheme A]", 42, 26, 152, 97)

9998

In [13]:
rt = random_team(() -> rand_rename(random_scheme_a()), 1000)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Scorpion Mlani,42,99,28,47
2,Dancing Zac,86,13,99,28
3,Kody the Prankster,84,33,72,35
4,Siaire the Shy,114,32,49,23
5,Xceeding Dog,38,98,28,53
6,Butler Cintia,36,130,17,36
7,Cayla the Penguin,67,45,70,51
8,Ingenious Martini,78,44,62,39
9,Adhira the Mushroom,114,34,47,22


In [14]:
counters = random_team(() -> pick_best_rdmly(library, rt, 9), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Heroic Cheese,55,54,59,70
2,Avagrace the Merciful,95,79,10,19
3,Militant Armadillo,71,64,12,71
4,Rough Pangolin,46,74,14,80
5,Powerful Gemstone,43,56,84,71
6,Super Ferdinand,18,99,16,80
7,Lenora the Icy,20,100,10,78
8,Joyful Elephant,47,40,91,96
9,Jesaiah the Creative,46,54,74,76


In [15]:
for f in rt
    v = eval_battle_list(f, counters)
    if v < -50
        println(f)
        println(v)
    end
end

Fighter("Scorpion Mlani", 42, 99, 28, 47)
-56
Fighter("Xceeding Dog", 38, 98, 28, 53)
-70
Fighter("Butler Cintia", 36, 130, 17, 36)
-70
Fighter("Creative Ema", 45, 14, 188, 65)
-64
Fighter("Landis the Mantis", 42, 89, 32, 55)
-66
Fighter("Xceeding Jellyfish", 36, 23, 179, 118)
-52
Fighter("Slingshot Isan", 36, 80, 40, 71)
-58
Fighter("Desi the Hungry", 33, 103, 25, 56)
-64
Fighter("Tyrael the Soldier", 33, 111, 23, 50)
-70
Fighter("California the Lawyer", 41, 77, 41, 67)
-68
Fighter("Candle Evyana", 47, 20, 161, 74)
-56
Fighter("Invincible Kahealani", 35, 23, 182, 122)
-52
Fighter("Leora the Knight", 22, 107, 21, 67)
-54
Fighter("Eastyn the Doctor", 29, 105, 26, 59)
-62
Fighter("Rosmery the Warthog", 30, 156, 16, 31)
-70
Fighter("Archer Zamarian", 42, 84, 36, 59)
-66
Fighter("Little Samurai", 43, 83, 35, 59)
-62
Fighter("Aryeh the Kitten", 29, 20, 218, 154)
-54
Fighter("Itinerant Marksman", 38, 105, 25, 48)
-70
Fighter("Robot Coumba", 33, 127, 18, 41)
-70
Fighter("Iguana Zalia", 44, 89

Fighter("Leoni the Armadillo", 43, 75, 42, 66)
-64
Fighter("Victorious Spider", 37, 111, 24, 45)
-70
Fighter("Engine Haven", 29, 21, 210, 157)
-54
Fighter("Heron Zahi", 26, 128, 20, 48)
-70
Fighter("Seasoned Yardstick", 42, 98, 28, 48)
-58
Fighter("Studious Ilse", 33, 82, 37, 74)
-52
Fighter("Ancient Adrit", 28, 149, 16, 36)
-70
Fighter("Irah the Xcecutive", 41, 113, 20, 40)
-56
Fighter("Atmospheric Samurai", 35, 83, 36, 70)
-70
Fighter("Warm Urchin", 33, 90, 33, 66)
-74
Fighter("Renee the Itinerant", 47, 94, 28, 45)
-52
Fighter("Neviah the Gamer", 33, 101, 27, 57)
-64
Fighter("Tenley the Triangle", 34, 112, 24, 48)
-70
Fighter("Arya the Superficial", 41, 96, 28, 51)
-66
Fighter("Pig Keyler", 44, 87, 33, 54)
-64
Fighter("Bona the Zookeeper", 42, 96, 27, 50)
-64
Fighter("Royal Octopus", 34, 87, 33, 68)
-72
Fighter("Makynzie the Doctor", 43, 80, 39, 61)
-62
Fighter("Keivon the Iguana", 31, 97, 28, 63)
-66
Fighter("Lucid Hue", 25, 148, 15, 40)
-70
Fighter("Beautiful Nur", 46, 19, 167, 76)

In [16]:
for f in rt
    v = eval_battle_list(f, counters)
    if v > 30
        println(f)
        println(v)
    end
end

Fighter("Dancing Zac", 86, 13, 99, 28)
52
Fighter("Blind Narjis", 81, 11, 109, 25)
34
Fighter("Yak Brizeida", 32, 56, 67, 108)
40
Fighter("Jb the Orangutan", 33, 64, 54, 95)
38
Fighter("Mikey the Bee", 16, 70, 38, 118)
58
Fighter("Sassy Blueberry", 88, 18, 90, 27)
36
Fighter("Classy Ox", 23, 71, 42, 104)
60
Fighter("Fireball Zendayah", 22, 78, 36, 96)
56
Fighter("Triangle Ramaya", 23, 65, 47, 114)
54
Fighter("Kingly Cat", 33, 59, 61, 102)
46
Fighter("Warm Car", 20, 73, 36, 107)
62
Fighter("Keita the Yak", 16, 64, 41, 130)
54
Fighter("Maximal Trey", 27, 58, 61, 117)
42
Fighter("Lior the Powerful", 15, 79, 29, 106)
70
Fighter("Ladder Zavannah", 24, 65, 48, 112)
54
Fighter("Old Warrior", 22, 66, 46, 114)
54
Fighter("Insane Lawyer", 15, 67, 36, 126)
58
Fighter("Popular Pigeon", 89, 16, 92, 24)
48
Fighter("Daring Robot", 28, 71, 45, 95)
44
Fighter("Egotistical Piano", 27, 70, 46, 98)
56
Fighter("Gallant Nightingale", 33, 64, 54, 95)
38
Fighter("Amayrah the Sombrero", 90, 13, 94, 28)
56
Figh

In [17]:
evaluate_generator(random_scheme_a, 100, 20)

-0.43900000000000006

In [18]:
evaluate_generator(random_fighter, 100, 20)

-0.859

In [19]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.8470000000000002

In [20]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.8500000000000002

In [21]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.8280000000000001

In [22]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.7939999999999999

## Team battles

In [23]:
team_a = random_team(() -> rand_rename(random_scheme_a()), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Ante the Rogue", 85, 31, 70, 45)
 Fighter("Atmospheric Avaeya", 82, 73, 21, 31)
 Fighter("Elusive Artisan", 47, 87, 66, 32)
 Fighter("Uber Monkey", 52, 64, 20, 88)
 Fighter("Original Lion", 65, 42, 79, 50)
 Fighter("Archer Gavin", 29, 98, 84, 48)
 Fighter("Taleah the Zookeeper", 65, 93, 45, 11)
 Fighter("Aggressive Blanket", 87, 48, 52, 27)
 Fighter("Umbrella Godric", 58, 79, 78, 11)
 Fighter("Orange Eliahana", 72, 34, 99, 12)
 Fighter("Good Xerox", 25, 88, 100, 60)
 Fighter("Hunter Loai", 42, 93, 14, 59)
 Fighter("Tevon the Zipper", 55, 89, 26, 41)
 ⋮
 Fighter("Anubis the Ingenious", 76, 26, 96, 28)
 Fighter("Valiant Cake", 46, 61, 100, 42)
 Fighter("Nyair the Commander", 45, 80, 37, 59)
 Fighter("Marmot Lakeisha", 75, 39, 86, 16)
 Fighter("Frozen Sammie", 65, 23, 97, 95)
 Fighter("Quiet Mosquito", 72, 75, 40, 22)
 Fighter("Vacuum Justo", 23, 98, 25, 73)
 Fighter("Pie Marjorie", 69, 24, 97, 68)
 Fighter("Breylan the Butler", 72, 77, 44, 16)
 Figh

In [24]:
eval_team_battle(team_a, counters)

-1

In [25]:
eval_team_battle(team_a, team_r0)

1

In [26]:
eval_team_battle(team_a, team_r1)

1

In [27]:
eval_team_battle(team_a, team_r2)

1

In [28]:
eval_team_battle(team_r0, team_a)

-1

In [29]:
eval_team_battle(team_r0, team_r1)

1

In [30]:
eval_team_battle(team_r0, team_r2)

1

In [31]:
as = counters
bs = team_a
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Heroic Cheese", 55, 54, 59, 70)
         Fighter("Invincible Ox", 36, 60, 61, 94)
Heroic Cheese lost against Invincible Ox
         Fighter("Avagrace the Merciful", 95, 79, 10, 19)
Avagrace the Merciful won against Invincible Ox
         Fighter("Chihuahua Oakleigh", 140, 33, 34, 18)
Avagrace the Merciful lost against Chihuahua Oakleigh
         Fighter("Militant Armadillo", 71, 64, 12, 71)
Militant Armadillo lost against Chihuahua Oakleigh
         Fighter("Rough Pangolin", 46, 74, 14, 80)
Rough Pangolin lost against Chihuahua Oakleigh
         Fighter("Powerful Gemstone", 43, 56, 84, 71)
Powerful Gemstone won against Chihuahua Oakleigh
         Fighter("Monkey Ahryah", 41, 84, 34, 61)
Powerful Gemstone won against Monkey Ahryah
         Fighter("Xray Davidson", 25, 93, 28, 75)
Powerful Gemstone lost against Xray Davidson
         Fighter("Super Ferdinand", 18, 99, 16, 80)
Super Ferdinand won against Xray Davidson
         Fighter("Zanora the Jellyfish", 107, 34, 52,

         Fighter("Luminous Jacket", 47, 50, 71, 86)
Luminous Jacket won against Hungry Lawyer
         Fighter("Yellow Cobbler", 128, 34, 39, 19)
Luminous Jacket won against Yellow Cobbler
         Fighter("Oak Pug", 36, 73, 44, 79)
Luminous Jacket won against Oak Pug
         Fighter("Wonderous Gangster", 14, 50, 57, 170)
Luminous Jacket lost against Wonderous Gangster
         Fighter("Curious Stokely", 55, 74, 93, 11)
Curious Stokely won against Wonderous Gangster
         Fighter("Finnigan the Itinerant", 71, 52, 56, 44)
Curious Stokely won against Finnigan the Itinerant
         Fighter("Enthusiastic Zookeeper", 31, 82, 36, 77)
Curious Stokely lost against Enthusiastic Zookeeper
         Fighter("Lillah the Nutritious", 100, 12, 86, 16)
Lillah the Nutritious won against Enthusiastic Zookeeper
         Fighter("Elevated Eel", 27, 53, 70, 126)
Lillah the Nutritious won against Elevated Eel
         Fighter("Billy the Cobbler", 40, 20, 182, 96)
Lillah the Nutritious lost against Bill

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [32]:
a_i, b_i

(99, 101)